# 02 Decompression relay

In [1]:
# Shortcut for pre-run conditions

import numpy as np
import pyDEW
import os
from thermoengine import model
from copy import deepcopy
import pandas as pd
import linecache
from scipy.interpolate import LinearNDInterpolator
from timeit import default_timer as timer
import warnings
import shutil
import time

import pyDEW_functions as pDF
import relay_functions as rF

In [2]:
# pyDEW flags up a few things when the system is created - this counteracts it
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

# These are the additional functions which we need to work with pyDEW inputs/outputs
import pyDEW_functions as pDF

if not os.path.exists("./data"):
    os.makedirs("./data")

# Adjust here the basis species in the DEW system - deepcopy to avoid mucking up defaults
defaultBasisSpecies = pyDEW.defaultsystem.basis_species_names
basisSpecies = deepcopy(defaultBasisSpecies)

defaultOtherSpecies = pyDEW.defaultsystem.other_species_names
otherSpecies = deepcopy(defaultOtherSpecies)

# Add missing Mg complex to the other species
otherSpecies.append('MG(OH)2(AQ)')

# Switch basis species - replace CO2(AQ) with CO3-2 directly to avoid pyDEW issues
basisSpecies[basisSpecies.index('CO2(AQ)')] = 'CO3-2'

# Create DEW system
dew_system = pyDEW.System(basis_species=basisSpecies, other_species=otherSpecies)

# pyDEW minutia - to match Guillaume's setups
# Suppress H2CO3(AQ)
nxmods = [['H2CO3(AQ)', 0, -1, 0]]  # Shouldn't amtter according to Guillaume


In [3]:
# workingT = 400 +273
# workingP = 15000

# fluidComposition

In [4]:
# List of reactants formed from mineral dictionaries
olvDict = {'reactant': 'OLIVINE(SS)',
           'moles': 5.9,
           'composition': {'FORSTERITE': 0.9, 'FAYALITE': 0.1}}

opxDict = {'reactant': 'ORTHOPYROXENE(SS)',
           'moles': 1.9,
           'composition': {'ENSTATITE_OR': 0.9, 'FERROSILITE': 0.1}}

cpxDict = {'reactant': 'CLINOPYROXENE(SS)',
           'moles': 0.2,
           'composition': {'DIOPSIDE': 0.9, 'HEDENBERGITE': 0.1}}

spnDict = {'reactant': 'SPINEL',
           'moles': 0.04}

mgtDict = {'reactant': 'MAGNETITE',
           'moles': 0.04}

startingReactant = [olvDict, opxDict, cpxDict, spnDict, mgtDict]
startingReactant = pDF.reactantToMass(startingReactant, 1000.0, dew_system)
startingReactantDF = pDF.reactantToDF(startingReactant, dew_system)
# volumeCheck = pDF.reactantToVolDF(startingReactant, dew_system, 10000, 600).iloc[0].to_dict()
# print(volumeCheck)
#pDF.volReactantToVolDF(volumeCheck, dew_system, 10000, 600)

In [5]:
temperatures = np.linspace(300+273.15, 300+273.15, 3)
pressures = np.linspace(15000.0, 14000.0, 3)
startingComposition = pd.concat([startingReactantDF]*len(pressures))
startingComposition['Pressure'] = pressures
startingComposition['Temperature'] = temperatures
# startingComposition

fluidComposition = rF.createHybridFluid([0,0,1], 15000.0, 300+273.15)
# fluidComposition[1]

In [6]:
# workingFluid = pyDEW.Fluid(dew_system, workingT, workingP, molalities=fluidComposition[0], fO2=fluidComposition[1],
#                            uebal='H+', uacion='CL-', nxmods=nxmods, aH2O_mode='molfraction')
# # rF.reactionByMass(startingReactant, workingFluid, 1000.0, dew_system, workingP, workingT)
# workingFluid.fO2

## Decompression relay mark 2

In [7]:
def decompressionRelay(startingColumn, startingFluid, startingfO2, system, timesteps=10, startingFRR=1.0, terminalFRR=0.1,
                       molalityLimit=-20.0, exportPath='data', outputToRead='working/output', tabToRead='working/tab', **kwargs):
    print('Time loop started')
    t = 0
    allCombinedOutputs = []
    while t < timesteps:
        if t == 0:
            workingColumn = startingColumn
        else:
            workingColumn = lastColumn

        # To keep track of errors. At n = 0, this is False
        errorFlag = False
        FRFlag = False
        passFlag = False
        calculations = 0
        passes = 0
        errors = 0
        outputs = []

        ##### DECOMPRESSION RELAY #####
        for n in range(len(workingColumn)):
            # shutil.rmtree('working')
            # time.sleep(3.0)
            workingParameters = workingColumn.iloc[n]
            workingReactant = pDF.DFtoReactant(workingParameters, system)
            workingT = workingParameters['Temperature']
            workingP = workingParameters['Pressure']

            if n == 0:
                workingFRR = startingFRR
                if t == 0:
                    slabFluidMass = startingFRR * pDF.findReactantMass(workingReactant, system)
                workingFluidMass = slabFluidMass
                workingFluidMolalities = startingFluid
                workingfO2 = startingfO2

            else:
                reactionInput = deepcopy(reactionOutput)
                workingFRR = reactionInput['Properties']['final FRR']
                workingFluidMass = reactionInput['Properties']['final fluid mass']
                workingfO2 = reactionInput['Properties']['final fluid fO2']
                if errorFlag == True:
                    workingFluidMolalities = reactionInput['Elements']
                    errorFlag = False
                else:
                    workingFluidMolalities = rF.formatFluid(reactionInput['Elements'])
                workingFluidMolalities.update((i, 10**j) for i, j in workingFluidMolalities.items())
                for i in workingFluidMolalities:
                    if workingFluidMolalities[i] < 10**(-20.0):
                        workingFluidMolalities[i] = 0.0
                previousSpecies = reactionInput['Species']
            workingFluid = pyDEW.Fluid(system, workingT, workingP, molalities=workingFluidMolalities, fO2=workingfO2,
                                       uebal='H+', uacion='CL-', nxmods=nxmods, aH2O_mode='molfraction')
            try:
                if workingFRR < terminalFRR:
                    FRFlag = True
                    print('Fluid-rock ratio below threshold at [n=' + str(n) +', t=' + str(t) + '], raising Error')
                    errorMessage = 'F/R threshold (' + str(terminalFRRatio) + ') achieved, using fluid/solid results from previous n/t'
                    raise pyDEW.core.RunError('F/R too low')
                reactionOutput = rF.reactionByMass(workingReactant, workingFluid, workingFluidMass, system, workingP, workingT)

            except (KeyError, pyDEW.core.RunError):   # (KeyError, NameError, pyDEW.core.RunError):
                errorFlag=True
                reactionOutput = rF.failedReactionOutput(workingReactant, workingFluid, workingFluidMass, system, workingP, workingT)
            outputs.append(reactionOutput)
        combinedOutputs = rF.combineOutputs(outputs, system)
        allCombinedOutputs.append(combinedOutputs)
        if timesteps - t > 1:
            lastColumn = deepcopy(combinedOutputs[1])
            lastColumn['Temperature'] = combinedOutputs[0]['Temperature (K)'].to_list()
            lastColumn['Pressure'] = combinedOutputs[0]['Pressure (bar)'].to_list()
            t+=1
        else:
            rF.combineTimes(allCombinedOutputs)
            print('Complete')
            return

In [8]:
decompressionRelay(startingComposition, fluidComposition[0], fluidComposition[1], dew_system, timesteps=3)

Time loop started
pyDEW index
167
pyDEW lz
0.2279
Output index
180
Output lz
0.2278867
pyDEW len fO2
181
kev len fO2
181
kev fO2
[-33.277, -33.277, -33.277, -33.277, -33.277, -33.277, -33.277, -33.277, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2769, -33.2768, -33.2768, -33.2768, -33.2768, -33.2768, -33.2768, -33.2767, -33.2767, -33.2767, -33.2766, -33.2766, -33.2766, -33.2765, -33.2765, -33.2764, -33.2763, -33.2762, -33.2762, -33.2761, -33.276, -33.2758, -33.2757, -33.2755, -33.2754, -33.2752, -33.2749, -33.2747, -33.2744, -33.2741, -33.2738, -33.2734, -33.2729, -33.2724, -33.2719, -33.2713, -33.2706, -33.2698, -33.2689, -33.2679, -33.2667, -33.2655, -33.264, -33.2624, -33.2606, -33.2589, -33.2585, -33.2561, -33.2533, -33.2502, -33.2467, -33.2427, -33.2382, -33.233, -33.2271, -33.2203, -33.2125, -33.2035, -33.193, -3